In [1]:
import molsysmt as msm
import numpy as np
from molsysmt import pyunitwizard as puw

In [5]:
molsys_AF = msm.convert('AF-P52789-F1')

In [10]:
molsys_AF = msm.build.add_missing_hydrogens(molsys_AF, pH=7.4)

In [11]:
import openmm as mm
from openmm import unit
from openmm import app
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt

In [13]:
topology = msm.convert(molsys_AF, to_form='openmm.Topology')
positions = msm.get(molsys_AF, element='atom', structure_indices=0, coordinates=True)
positions = msm.pyunitwizard.convert(positions[0], to_form='openmm.unit')

In [14]:
forcefield = app.ForceField("amber14-all.xml")

In [15]:
system = forcefield.createSystem(topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [16]:
temperature = 0*unit.kelvin

integration_timestep = 2.0*unit.femtoseconds
saving_interval = 1.00*unit.picoseconds
logging_interval = 100.00*unit.picoseconds
simulation_time = 500.*unit.picoseconds

saving_steps = int(saving_interval/integration_timestep)
logging_steps = int(logging_interval/integration_timestep)
md_steps = int(simulation_time/integration_timestep)

In [17]:
friction   = 1.0/unit.picoseconds
integrator = mm.LangevinIntegrator(temperature, friction, integration_timestep)

In [18]:
platform = mm.Platform.getPlatformByName('OpenCL')

In [19]:
simulation = app.Simulation(topology, system, integrator, platform)
simulation.context.setPositions(positions)

In [20]:
state=simulation.context.getState(getEnergy=True)
state.getPotentialEnergy()

Quantity(value=112825.046875, unit=kilojoule/mole)

In [22]:
simulation.context.getState(getPositions=True)

<openmm.openmm.State; proxy of <Swig Object of type 'OpenMM::State *' at 0x769e09a8a160> >

In [21]:
new_coordinates = msm.get(simulation, coordinates=True)

AttributeError: module 'molsysmt.form.openmm_Simulation' has no attribute 'get_coordinates_from_system'